In [ ]:
from pyannote.audio import Pipeline
import whisper
import torch
from pyannote.core import Segment
import time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")
print("Загрузка моделей...")
try:
    diarization_pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization-3.1",
        use_auth_token=""
    )
    diarization_pipeline.to(device)    
    whisper_model = whisper.load_model("turbo").to(device)    
    audio_file = "/kaggle/input/werty21222/audio.mp3"   
    
    print("\nВыполнение диаризации...")
    start_time = time.time()
    diarization = diarization_pipeline(audio_file)
    print(f"Диаризация выполнена за {time.time() - start_time:.2f} сек")    
    
    print("\nТранскрибация сегментов...")
    results = []
    
    audio = whisper.load_audio(audio_file) 
    
    for segment, _, speaker in diarization.itertracks(yield_label=True):
        start, end = segment.start, segment.end
        print(f"\nСпикер {speaker} ({start:.1f}-{end:.1f} сек):")        
         
        audio_segment = audio[int(start * 16000):int(end * 16000)]
        
        result = whisper_model.transcribe(audio_segment)
        text = result["text"]
        print(text)        
        results.append({
            "speaker": speaker,
            "start": start,
            "end": end,
            "text": text
        })
    
    print("\nФинальные результаты:")
    for item in results:
        print(f"[{item['start']:.1f}-{item['end']:.1f}] {item['speaker']}: {item['text']}")

except Exception as e:
    print(f"Произошла ошибка: {str(e)}")